In [1]:
import sys
import glob
import numpy as np
import os
from rdkit import Chem
import pandas as pd
from rdkit.Chem.PandasTools import LoadSDF
import matplotlib.pyplot as plt
import random
import networkx as nx
from networkx import Graph
from itertools import combinations
from rapidfuzz import process
from rapidfuzz.distance import JaroWinkler

In [25]:
df = pd.read_excel("10001_11911.xlsx" , engine="openpyxl")

/Users/danielruiz/opt/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [26]:
reagent = "3-chloro-benzenecarboperoxoic acid"
reagentList = df['Reagent']

In [27]:
refinedDF = df[df["Reagent"] == reagent].copy()
refinedDF.reset_index(drop=True, inplace=True)
selected_columns = ["Reaction ID", "Reaction", "Fulltext of reaction" , "Reactant","Yield (numerical)" , "Reagent" , "Catalyst" , "References"]  # Replace with actual column names

# Select only those columns
refinedDF = refinedDF[refinedDF['Reaction'].notna()].copy()
refinedDF = refinedDF[selected_columns].copy()

refinedDF = refinedDF[refinedDF['Yield (numerical)'].notna()].copy()
refinedDF.reset_index(drop=True, inplace=True)

print(refinedDF.shape)

(1415, 8)


In [28]:
refinedDF

,Reaction ID,Reaction,Fulltext of reaction,Reactant,Yield (numerical),Reagent,Catalyst,References
0,53907096,C\C=C/C=C/CO[Si](C1=CC=CC=C1)(C1=CC=CC=C1)C(C)...,NaN,C22H28OSi,45; 23,3-chloro-benzenecarboperoxoic acid,NaN,"Article; Tjeng, Andy A.;Handore, Kishor L.;Bat..."
1,54148574,COC1=CC=C(C\C=C\CCCCOS(=O)(=O)C2=CC=C(C)C=C2)C...,NaN,(E)-7-(4-methoxyphenyl)hept-5-en-1-yl 4-methyl...,42,3-chloro-benzenecarboperoxoic acid,NaN,"Article; Xu, Shiyan; Holst, Hannah M.; McGuire..."
2,54148604,COC1=CC=C(C\C=C\CCCCN2C(=O)C3=C(C=CC=C3)C2=O)C...,NaN,(E)-2-(7-(4-methoxyphenyl)hept-5-en-1-yl)isoin...,42,3-chloro-benzenecarboperoxoic acid,NaN,"Article; Xu, Shiyan; Holst, Hannah M.; McGuire..."
3,54214388,OCCCC#CC(=C)CCO>>OCCCC#CC1(CCO)CO1,NaN,C9H14O2,42,3-chloro-benzenecarboperoxoic acid,NaN,"Article; Lee, Jaeyeon; Kim, Jiheon; Lee, Hee-Y..."
4,54379735,COC1=CC=C(C=C1)C(\N(CC1=CC=CC=C1)S(=O)(=O)C1=C...,NaN,(E)-N-benzyl-N-(1-(4-methoxyphenyl)-2-phenyl-2...,42,3-chloro-benzenecarboperoxoic acid,NaN,"Article; Dutta, Shubham;Yang, Shengwen;Vanjari..."
...,...,...,...,...,...,...,...,...
1410,2127842,[H][C@@]12CCC3=C(C=CC(OC)=C3)[C@@]1([H])CC[C@]...,NaN,"17β-hydroxy-3-methoxy-1,3,5(10),14-estratetraene",49; 24,3-chloro-benzenecarboperoxoic acid,NaN,"Article; Ponsold, Kurt; Schubert, Gerd; Wunder..."
1411,3201566,[H][C@@]12CCC3=C(C=CC(OC)=C3)[C@@]1([H])CC[C@]...,NaN,"3-Methoxy-estra-1,3,5(10),14-tetraen-17α-ol-ac...",53; 29,3-chloro-benzenecarboperoxoic acid,NaN,"Article; Ponsold, Kurt; Schubert, Gerd; Wunder..."
1412,3201566,[H][C@@]12CCC3=C(C=CC(OC)=C3)[C@@]1([H])CC[C@]...,NaN,"3-Methoxy-estra-1,3,5(10),14-tetraen-17α-ol-ac...",53; 29,3-chloro-benzenecarboperoxoic acid,NaN,"Article; Ponsold, Kurt; Schubert, Gerd; Wunder..."
1413,3720181,O1[C@@H]2C=CC3=C(C=CC=N3)[C@H]12.O1[C@@H]2C=CC...,NaN,"quinoline 5,6-arene oxide",82,3-chloro-benzenecarboperoxoic acid,NaN,"Article; Boyd, Derek R.; Davies, R. Jeremy H.;..."


In [18]:
#from given dataset, group 
def groupBy(df, groupVar):
    parameters = np.array(df[groupVar])
    parameters = np.unique(parameters)
    dfMAST = []
    
    for i in range (len(parameters)):
        parm = parameters[i]
        smallDF = df[df[groupVar] == parm].copy()
        smallDF.reset_index(drop=True, inplace=True)
        dfMAST.append(smallDF)
    return dfMAST



In [29]:
partitionedDF = groupBy(refinedDF, "Reaction ID")

In [30]:
print(len(partitionedDF))


1355


In [31]:
columnsMAST = ["ID" , "SMILES" , "Yield" ,"Catalyst" ,  "Source"]
masterDF = pd.DataFrame(columns=columnsMAST)
for j in range (len(partitionedDF)):
    smallDF = partitionedDF[j]
    reactionType = list(smallDF['Reaction'])
    char = "."
    found = any(char in string for string in reactionType)
    #print("found" , found)
    if not found:
        #print("newMolec")
        #goal is to group by catalyst type first before averaging the yield data 
        
        catalystType = smallDF['Catalyst']
        newCatList = np.where(pd.isna(catalystType), "None", catalystType)
        yields = list(smallDF["Yield (numerical)"])
        print("YIELDS:" , yields)
        yieldsListMAST = []
        yieldIndexList = []
        for i in range (len(yields)):
            try:
                yield_ = float(yields[i])
                yieldsListMAST.append(yield_)
                yieldIndexList.append(i)
            except ValueError:
                print("ValueError" ,i ,  yield_)
                            
                        
        if len(yieldIndexList) != 0:
            maxValue = max(yieldsListMAST)
            if maxValue  <= 100.00:
                #print("yieldsListMAST:", yieldsListMAST)
                #print("yieldIndexList:", yieldIndexList)
                maxIndices = [yieldIndexList[i] for i, val in enumerate(yieldsListMAST) if val == maxValue]
                #print("max Index" , maxIndices)
                targetInd = maxIndices[0]
                newIndexList = [targetInd]
                #print("targetInd" , targetInd)
                #print("newCatList" , newCatList)
                finalCat = newCatList[targetInd]
                for i, catalyst in enumerate (newCatList):
                    #print(catalyst)
                    if i != targetInd:
                        #print("pass")
                        similarity = JaroWinkler.similarity(catalyst, finalCat)
                        #print("similarity" , similarity)
                        if similarity >= 0.75:
                            newIndexList.append(i)
                newYieldsList = []
                #print("newIndexList" , newIndexList)
                #print(yieldsListMAST)
                for i, ind in enumerate(newIndexList):
                    newYieldsList.append(yieldsListMAST[ind])
                finalYield = np.mean(newYieldsList)
                reactionType = list(smallDF['Reaction'])
                reaction = reactionType[targetInd]
                smiles = str(reaction.split(">")[0])
                refs = list(smallDF["References"])
                ref = refs[targetInd]
                id_ = list(smallDF["Reaction ID"])
                idMAST = id_[targetInd]
                masterDF.loc[len(masterDF)] = [idMAST , smiles, finalYield , finalCat , ref]
                

YIELDS: ['41']
YIELDS: ['49']
YIELDS: ['71']
YIELDS: ['83']
YIELDS: ['83']
YIELDS: ['81']
YIELDS: ['83']
YIELDS: ['78']
YIELDS: ['86']
YIELDS: ['78']
YIELDS: ['78']
YIELDS: ['78']
YIELDS: ['75']
YIELDS: ['85']
YIELDS: ['71']
YIELDS: ['71']
YIELDS: ['76']
YIELDS: ['49']
YIELDS: ['11']
YIELDS: ['40']
YIELDS: ['18']
YIELDS: ['35']
YIELDS: ['35']
YIELDS: ['84']
YIELDS: ['11']
YIELDS: ['85']
YIELDS: ['62']
YIELDS: ['10']
YIELDS: ['81']
YIELDS: ['59']
YIELDS: ['70']
YIELDS: ['72']
YIELDS: ['40']
YIELDS: ['24']
YIELDS: ['38']
YIELDS: ['78']
YIELDS: ['68']
YIELDS: ['22']
YIELDS: ['79']
YIELDS: ['67']
YIELDS: ['80']
YIELDS: ['57']
YIELDS: ['28']
YIELDS: ['59']
YIELDS: ['58']
YIELDS: ['30']
YIELDS: ['55']
YIELDS: ['39']
YIELDS: ['57']
YIELDS: ['44']
YIELDS: ['48']
YIELDS: ['60']
YIELDS: ['75']
YIELDS: ['20']
YIELDS: ['46']
YIELDS: ['47']
YIELDS: ['60.5']
YIELDS: ['80']
YIELDS: ['100']
YIELDS: ['55']
YIELDS: ['16']
YIELDS: ['71']
YIELDS: ['69']
YIELDS: ['46']
YIELDS: ['51']
YIELDS: ['55']
YIELDS:

In [32]:
masterDF.shape

(828, 5)

In [33]:
masterDF

,ID,SMILES,Yield,Catalyst,Source
0,10079066,C(OC1CCCCO1)[C@H]1COCC=C1,41.0,None,"Article; Banfi, Luca; Guanti, Giuseppe; Paravi..."
1,10081428,[H][C@@]12C[C@@H](CC(=O)OCC)C(=C)C[C@]1([H])C2...,49.0,None,"Article; Frackowiak, Bozena; Ochalik, Katarzyn..."
2,10160437,ClCC#CCCCC=C,71.0,None,"Article; Shen, Kuo-Hui; Lush, Shie-Fu; Chen, T..."
3,10160679,CC(=C)CCCC#CCCl,83.0,None,"Article; Shen, Kuo-Hui; Lush, Shie-Fu; Chen, T..."
4,10160727,COC\C=C/COCC#CCCl,83.0,None,"Article; Shen, Kuo-Hui; Lush, Shie-Fu; Chen, T..."
...,...,...,...,...,...
823,9859767,CCCCCC[C@H](C\C=C/CCCCCCCC(=O)NCC1=CC=C(O)C(OC...,35.0,None,"Article; Appendino, Giovanni; De Petrocellis, ..."
824,9885460,COC(=O)\C=C/C1=C(Br)C=CC=C1,24.0,None,"Article; Minin, Patricia L.; Walton, John C.; ..."
825,9907496,CC1(C)O[C@H]2CCC=C(Cl)[C@H]2O1,65.0,None,"Article; Fonseca, German; Seoane, Gustavo A.; ..."
826,9911160,CC1CC(=O)C(=C)[C@]11C[C@H](C)CC(C)(C)C1,61.0,None,"Article; Kraft, Philip; Eichenberger, Walter; ..."


In [34]:
masterDF.to_excel("mCPBA3.xlsx", index=False, engine="openpyxl")